### (2) 여러개의 변수 사용 (다 사용함)

In [0]:
from sklearn.datasets import load_diabetes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from sklearn.model_selection import train_test_split

In [0]:
diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target

In [0]:
target = diabetes['target']
data = (df).to_numpy()

data = diabetes['data']

# train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, shuffle=True, random_state=21)

In [0]:
print(x_train)

[[-0.08906294 -0.04464164 -0.04177375 ... -0.03949338  0.0011438
  -0.03007245]
 [-0.00188202 -0.04464164  0.06816308 ...  0.08670845  0.04613233
  -0.0010777 ]
 [-0.06363517 -0.04464164  0.03582872 ... -0.00259226 -0.02595242
  -0.05492509]
 ...
 [ 0.04170844 -0.04464164 -0.00836158 ... -0.0763945  -0.08023654
  -0.01764613]
 [-0.07453279 -0.04464164 -0.02345095 ... -0.03949338 -0.03845911
  -0.03007245]
 [ 0.0090156  -0.04464164  0.04552903 ... -0.03949338  0.04613233
   0.03620126]]


In [0]:
#train
a = pd.DataFrame(x_train, columns=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'])
b = pd.DataFrame(y_train, columns=['target'])
c = pd.concat([a, b], axis=1)

#test
d = pd.DataFrame(x_valid, columns=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'])
e = pd.DataFrame(y_valid, columns=['target'])
f = pd.concat([d, e], axis=1)

#train
df = spark.createDataFrame(c)
#test
test_df = spark.createDataFrame(f)

In [0]:
#train
assembler=VectorAssembler(inputCols=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'],outputCol='features')
output=assembler.transform(df)

#test
assembler=VectorAssembler(inputCols=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6'],outputCol='features')
test_data=assembler.transform(test_df)

In [0]:
#import LinearRegression library
from pyspark.ml.regression import LinearRegression
#creating an object of class LinearRegression
#object takes features and label as input arguments
ship_lr=LinearRegression(featuresCol='features',labelCol='target')
#pass train_data to train model
trained_ship_model=ship_lr.fit(output)
#evaluating model trained for Rsquared error
ship_results=trained_ship_model.evaluate(output)

# Summarize the model over the training set and print out some metrics
trainingSummary = trained_ship_model.summary

In [0]:
unlabeled_data=test_data.select('features')
predictions=trained_ship_model.transform(unlabeled_data)
pred = predictions.select('prediction')
pred_array = np.array(pred.collect())

### 결과

In [0]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print('Rsquared Error :', ship_results.r2)
print('Coefficients: ', trained_ship_model.coefficients)

RMSE: 51.784667
Rsquared Error : 0.5534645067406799
Coefficients:  [-4.745027116969422,-253.9133024670898,512.2737219724899,375.8299050060503,-921.4384001137577,628.6727251640656,183.3387174322452,247.06280135539848,783.3124246080519,16.711343108660717]
